## Regualr Returning Function

When calling for `for now in search_twitter("cat pictures"):` all the data must first be downloaded before the first record is returned

In [ ]:
def search_twitter(query):
	data = []
	for row in paginated_get(query):
		data.append(row)
	return data

# Collect all the cat picture data
for row in search_twitter("cat pictures"):
  # Once collected, 
  # print row by row
	print(row)

### Generator

we yield each row as we get the data, without collecting it into memory.

In [ ]:
def search_twitter(query):
	for row in paginated_get(query):
		yield row

# Get one row at a time
for row in extract_data("cat pictures"):
	# print the row
	print(row)
  # do something with the row such as cleaning it and writing it to a buffer
	# continue requesting and printing data

### Example 1: Grabbing data from an api

The api documentation is as follows:

 - There are a limited nr of records behind the api
 - The data can be requested page by page, each page containing 1000 records
 - If we request a page with no data, we will get a successful response with no data
 - so this means that when we get an empty page, we know there is no more data and we can stop requesting pages - this is a common way to paginate but not the only one - each api may be different.
 - details:
    - method: get
    - url: https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api
    - parameters: page integer. Represents the page number you are requesting. Defaults to 1.